# Practical 3: Supply chain linkages

### Objectives
- Understand the format of input-output tables
- Calculate the Leontief and Ghosh inverse
- Apply demand and price driven models
- Calculate backward and forward linkages

### Exercise 1
An IO system is provided with sectoral data aggregated in 3 sectors.
Use this IO data to calculate total product output and input, the Leontief quantity and price models, and the Ghosh model

In [1]:
# import modules
import pandas as pd
import numpy as np

1. Inter-industry transactions

In [2]:
labels = pd.Index(["Agriculture", "Manufacturing", "Services"])

Z = pd.DataFrame([
    [0.6,2.6,0.5],
    [0.8, 30.6, 7.8],
    [0.9,12.1,23]], index=labels, columns=labels)

Z

,Agriculture,Manufacturing,Services
Agriculture,0.6,2.6,0.5
Manufacturing,0.8,30.6,7.8
Services,0.9,12.1,23.0


2. Value added vector

In [3]:
# Value added (last row of the first table)
V = np.array([3.30, 22.4, 52.5])
V

array([ 3.3, 22.4, 52.5])

3. Final demand vector of products purchased by final consumers

In [4]:
Y = np.array([1.9, 28.5, 47.8])
Y

array([ 1.9, 28.5, 47.8])

### 1.1 Calculate total product outputs and inputs

4. calculate total outputs: intermediate output + final product output

    $x_i = \sum_{i=0}^n Z_i + Y_i$

    tip: use the .sum() method and remember that axis=0 is columns and axis=1 is rows

In [5]:
Z_sum = Z.sum(axis=1) 
x_out = Z_sum + Y
x_out

Agriculture       5.6
Manufacturing    67.7
Services         83.8
dtype: float64

5. calculate the total inputs: intermediate input + value added

    $x_j = \sum_{j=0}^n Z_j + V_j$

In [6]:
Z_sum = Z.sum(axis=0)
x_in = Z_sum + V
x_in

Agriculture       5.6
Manufacturing    67.7
Services         83.8
dtype: float64

### 1.2 Calculate the Leontief quantity and price models

1. Create a technical coefficient matrix and its identity matrix
    
    $\mathbf{A} = \mathbf{Z} \hat{\mathbf{x}}^{-1} $

    tip: 
    - use the @ operand or np.matmul() method for multiplications
    - use the numpy diag() method to diagonalize a vector
    - use numpy linalg.inv() method to perform inverses
    - use numpy identity() method for the identity matrix, tip: use the number of columns between the brackets

In [7]:
# Create the technical coefficient matrix A first
A = Z @ np.linalg.inv(np.diag(x_out))
A

,0,1,2
Agriculture,0.107143,0.038405,0.005967
Manufacturing,0.142857,0.451994,0.093079
Services,0.160714,0.178730,0.274463


In [8]:
# Create an identity matrix the same order (i.e., shape) as A
I = np.identity(A.shape[0]) # A.shape[0] is the same as writing the number of available sectors .shape allows you to see the dimensions of the dataframe

print(f"The dimension/order of I is {I.shape}")
I

The dimension/order of I is (3, 3)


array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

2. Calculate and verify the Leontief inverse

    $\mathbf{L} = (\mathbf{I}-\mathbf{A})^{-1}$

    Steps: 

    - Calculate Leontief inverse using numpy.linalg.inv method

    - multiply the Leontief inverse by final demand vector

In [9]:
# Calculate the leontief inverse matrix in the Demand-pull model
# Use numpy linear algebra inverse function to get L = (I - A)^-1
L = np.linalg.inv(I - A)
L

array([[1.1374997 , 0.08638219, 0.02043636],
       [0.35414416, 1.93137726, 0.25068774],
       [0.33920879, 0.49491248, 1.44457108]])

In [10]:
# Check that the total product output is equal to the one we have already calculated
x = L @ Y # @ operand is the same as np.dot(L, Y)
x
# beware when using np.matmul and np.dot/@ interchangeably. 
# for more info https://www.delftstack.com/howto/numpy/numpy-dot-vs-matmul/ 

array([ 5.6, 67.7, 83.8])

3. Calculate the Leontief price model

    $\mathbf{L} = (\mathbf{I}-\mathbf{A}')^{-1}$

    Steps: 

    - Transpose A

    - Calculate Leontief inverse using numpy.linalg.inv method

    - Calculate value added coefficients by dividing V by x
        
    - multiply the Leontief inverse by the value added coefficients

In [11]:
# Transpose the direct requirement matrix A
A_p = A.T
# Calculate the leontief inverse matrix in the price model
L_p = np.linalg.inv(I - A_p)
L_p

array([[1.1374997 , 0.35414416, 0.33920879],
       [0.08638219, 1.93137726, 0.49491248],
       [0.02043636, 0.25068774, 1.44457108]])

In [12]:
# calculate the value added coefficients by dividing the value added by total product output
v = V/x

# Check that p=1
p = L_p @ v
p

array([1., 1., 1.])

### 1.3 Calculate the Ghosh quantity model

1. Create a technical coefficient matrix B and its identity matrix

    $\mathbf{B} = \hat{\mathbf{x}}^{-1} \mathbf{Z}$

In [13]:
# Calculate B
B = np.linalg.inv(np.diag(x)) @ Z

# Create an identity matrix the same order (i.e., shape) as B
I = np.identity(3)
B

,Agriculture,Manufacturing,Services
0,0.107143,0.464286,0.089286
1,0.011817,0.451994,0.115214
2,0.010740,0.144391,0.274463


2. Calculate and verify Ghosh inverse

    $\mathbf{G} = (\mathbf{I}-\mathbf{B})^{-1}$

In [14]:
# Calculate the Ghosh inverse matrix
G = np.linalg.inv(I - B)
G

array([[1.1374997 , 1.04429896, 0.30581559],
       [0.02929405, 1.93137726, 0.31030477],
       [0.02266789, 0.39982786, 1.44457108]])

In [15]:
# Check that the total product input is equal to the one we have already calculated x=G'V
x_g = G.T @ V
x_g

array([ 5.6, 67.7, 83.8])

## Exercise 2
Calculate backward and forward linkages

### 2.1 Calculate backward linkages using A and L

1. First assign all labels to the numpy array by turning them into pandas dataframes

In [16]:
# Labelling Leontief quantity model
A.columns = labels
L = pd.DataFrame(L, index=labels, columns=labels)
print(f"A matrix:\n\n{A}\n\nL matrix:\n\n{L}")

A matrix:

               Agriculture  Manufacturing  Services
Agriculture       0.107143       0.038405  0.005967
Manufacturing     0.142857       0.451994  0.093079
Services          0.160714       0.178730  0.274463

L matrix:

               Agriculture  Manufacturing  Services
Agriculture       1.137500       0.086382  0.020436
Manufacturing     0.354144       1.931377  0.250688
Services          0.339209       0.494912  1.444571


2. Column-wise sum of of A and L

    tip: use .sum() method
    
    remember: in pandas axis=0 refers to columns axis=1 refers to rows

In [17]:
# Sum the sectors in A to obtain the direct backward linkages
s_a_sum = A.sum(0)

In [18]:
# Sum the sectors in L to obtain the total backward linkages
s_l_sum = L.sum(0)

3. Normalize backward linkages results

    steps:

    - find the average value of the linkages by dividing their total sum by the number of sector/products
    - divide the total linkages by the average value of the linkages 

In [19]:
# Normalize direct backward linkages results 
no_sectors = len(s_a_sum)
avg_a = s_a_sum.sum()/no_sectors
norm_dir_back_link = s_a_sum/avg_a
s_a_sum

Agriculture      0.410714
Manufacturing    0.669129
Services         0.373508
dtype: float64

In [20]:
# Normalize total backward linkages results 
avg_l = s_l_sum.sum()/no_sectors
norm_tot_back_link = s_l_sum/avg_l

In [21]:
# Assemble a table with direct and total backward linkages
norm_backward_linkages = pd.concat([norm_dir_back_link, norm_tot_back_link], axis=1)
norm_backward_linkages.columns = ["direct_back_link", "total_back_link"]
norm_backward_linkages

,direct_back_link,total_back_link
Agriculture,0.847794,0.906479
Manufacturing,1.381212,1.244057
Services,0.770994,0.849463


### 2.2 Calculate forward linkages using B and G

1. Assign labels to B and G

In [22]:
# Labelling Ghosh quantity model
B.index = labels
G = pd.DataFrame(G, index=labels, columns=labels)

2. Perform a row sum of B and G

    tip: use .sum() method

In [23]:
# Sum the products in B to obtain the direct forward linkages
s_b_sum = B.sum(1)
s_b_sum

Agriculture      0.660714
Manufacturing    0.579025
Services         0.429594
dtype: float64

In [24]:
# Sum the products in G for the total forward linkages
s_g_sum = G.sum(1)
s_g_sum

Agriculture      2.487614
Manufacturing    2.270976
Services         1.867067
dtype: float64

3. Normalize forward linkages results

    steps:

    - find the average value of the linkages by dividing their total sum by the number of sector/products
    - divide the total linkages by the average value of the linkages 

In [25]:
no_products = len(s_b_sum)

# Normalize direct forward linkages results 
avg_b = s_b_sum.sum()/no_products
norm_dir_fwd_link = s_b_sum/avg_b

In [26]:
# Normalize total forward linkages results 
avg_g = s_g_sum.sum()/no_products
norm_tot_fwd_link = s_g_sum/avg_g

In [27]:
# Assemble a table with direct and total forward linkages
norm_forward_linkages = pd.concat([norm_dir_fwd_link, norm_tot_fwd_link], axis=1)
norm_forward_linkages.columns = columns=["direct_fwd_link", "total_fwd_link"]
norm_forward_linkages

,direct_fwd_link,total_fwd_link
Agriculture,1.187386,1.126355
Manufacturing,1.040580,1.028265
Services,0.772034,0.845380


#### 2.3  Concatenate the results into one table

In [28]:
# Print normalized results for backward and forward linkages
pd.concat([norm_backward_linkages, norm_forward_linkages], axis=1)

,direct_back_link,total_back_link,direct_fwd_link,total_fwd_link
Agriculture,0.847794,0.906479,1.187386,1.126355
Manufacturing,1.381212,1.244057,1.040580,1.028265
Services,0.770994,0.849463,0.772034,0.845380


#### 2.4 Using the normalized results and the figure below, what can you tell about this IO system?

![image](./figures/linkages.PNG)

*Double click on the cell to replace this text with your answer*